In [24]:
from ipywidgets import interact, Textarea
import nltk
from colored import fg

import kiwi
from kiwi import constants as const

In [25]:
model_file = 'en_de.torch'
model = kiwi.load_model(model_file)



In [27]:
from IPython.display import Markdown, display
nltk.download('punkt')

def printmd(text):
    display(Markdown(text))
    
def make_color(text, color):
    start = "<span style='color:" + color + "'>"
    return start + text + "</span>"


def get_color(bad_prob):
    return 'green' if bad_prob < THRESHOLD else 'red'



[nltk_data] Downloading package punkt to /home/sony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:


def highlight(text_bad_prob):
    text, bad_prob = text_bad_prob
    if bad_prob >= THRESHOLD:
        text = '*' + text + '*'
    return make_color(text, get_color(bad_prob))


def KiwiViz(source, mt):
    source = ' '.join(nltk.word_tokenize(source))
    mt = ' '.join(nltk.word_tokenize(mt))
    model_out = model.predict({const.SOURCE: [source.lower()], const.TARGET: [mt.lower()]})
    bad_probs = model_out[const.TARGET_TAGS][0]
    gap_probs = model_out[const.GAP_TAGS][0]
    highlight_words = list(map(highlight, zip(mt.split(), bad_probs)))
    visualization = ''
    BAD_GAP = make_color('_', 'red')
    for gap_prob, word in zip(gap_probs[:-1], highlight_words):
        visualization += ' ' if gap_prob < THRESHOLD else BAD_GAP + ' '
        visualization += word
    visualization += '' if gap_probs[-1] < THRESHOLD else ' ' + BAD_GAP
    print('HTER: {}'.format(model_out[const.SENTENCE_SCORES][0]))
    printmd(visualization)

In [38]:
#SOURCE = Textarea(value='stops validating after detecting 250 errors.')
#MT = Textarea(value='validieren nach Entdeckung 250 Fehler stoppt.')
SOURCE = Textarea(value='the Sharpen tool sharpens areas in an image.')
MT = Textarea(value='der Schärfen-Werkezug Bereiche in einem Bild schärfer erscheint.')

In [39]:
_interact = interact(KiwiViz, source=SOURCE, mt=MT)

interactive(children=(Textarea(value='the Sharpen tool sharpens areas in an image.', description='source'), Te…